<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">F_Non_cadastre</font></strong></p>

WALOUS_UTS - Copyright (C) <2020> <Université catholique de Louvain (UCLouvain), Belgique;
					 Université Libre de Bruxelles (ULB), Belgique;
					 Institut Scientifique de Service Public (ISSeP), Belgique;
					 Service Public de Wallonie (SWP), Belgique >
						 							
	
List of the contributors to the development of WALOUS_UTS: see LICENSE file.


Description and complete License: see LICENSE file.
	
This program (WALOUS_UTS) is free software:
you can redistribute it and/or modify it under the terms of the
GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option)
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program (see COPYING file).  If not,
see <http://www.gnu.org/licenses/>.

---------
Jupyter Notebook containing the preprocessing steps consisting of: 
- Classification of uncadastred areas
- Merge table with classified cadastral geometries with classified uncadastred geometries into one single table

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Define working environment

**Import libraries**

In [ ]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob

In [ ]:
## Import Psycopg2 library (interection with postgres database)
import psycopg2
## Import Subprocess
import subprocess

In [ ]:
# Libraries for parallel processing
import multiprocessing
from multiprocessing import Pool
from functools import partial

In [ ]:
## Import Pandas library (View and manipulaiton of tables)
import pandas as pd
pd.set_option('display.max_columns', 100)

**Add folder with SCR provided belong to this notebook**

In [ ]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [ ]:
run ../SRC/config.py

In [ ]:
[print("%s: %s"%(k,config_parameters[k])) for k in config_parameters]

In [ ]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [ ]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

**Other functions**

In [ ]:
# Import functions for processing time information
import time
from processing_time import start_processing, print_processing_time
# Import function that check and create folder
from mkdir import check_create_dir

**Custom functions: Psycopg2 and Postgresql functions**

In [ ]:
# Import function that display postgresql's table header
from display_header import display_header
# Import function to creation connection to Postgresql database 
from postgres_functions import create_pg_connexion
# Import function to creation of Postgresql schema 
from postgres_functions import create_pg_schema
# Import function to give rights to user on a specific schema
from postgres_functions import grant_user
# Import function to add serial primary key to table
from postgres_functions import add_serial_pkey
# Import function to create index
from postgres_functions import create_index

In [ ]:
# Import function that fix not valid geometries
from postgres_geom import make_valid

In [ ]:
# Import function that manage importation of a Shapefile into postgresql database
from postgres_import import ogr2ogr_import
# Import a function that manage importation of a CSV file
from postgres_import import import_csv
# Import function to export postgis table as shapefile or geopackage
from postgres_export import ogr2ogr_export
# Import functions for creation of postgresql back-up and restoration
from dump_functions import dump_table, dump_db, restore_psql

In [ ]:
# Import function to create columns with walousmaj at each level 
from postgres_postclassification import create_walousmaj_levels
# Import function to create column 'hilucslanduse_1' corresponding to scenario 1 of INSPIRE HILUCS 
from postgres_postclassification import create_hilucs_landuse_1
# Import function to create column 'hilucslanduse_1' corresponding to scenario 1 of INSPIRE HILUCS 
from postgres_postclassification import create_hilucs_landuse_2

In [ ]:
# Import functions for creation of table with all geometries together
from postgres_postclassification import create_cuswall_table

**GRASS GIS Python libraries**

In [ ]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

In [ ]:
# Import function for GRASS GIS mapset checking and launching
from grass_database import check_gisdb, check_location, check_mapset, working_mapset, launch_mapset
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for importation of r.zonal.classes csv output into a GRASS GIS table
from grass_processing import rzonalclasses_sql_insert

# Import uncadastred data into PostgreSQL 

## Create new schema

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'non_cadastre', overwrite=False)
grant_user(con, 'non_cadastre', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Import file with administratives limits (communes)

In [ ]:
# Import file into postgis database
ogr2ogr_import(data['communes_rw'][1], config_parameters, 
               schema='non_cadastre', table=data['communes_rw'][0], to_srid='31370', 
               create_opt='-overwrite', verbose=True) 

## Import file with uncadastred geometries

In [ ]:
# Import file into postgis database
ogr2ogr_import(data['uncadastred_spaces'][1], config_parameters, 
               schema='non_cadastre', table=data['uncadastred_spaces'][0], to_srid='31370', 
               create_opt='-overwrite', verbose=True) 

## Dissolve adjacent geometries with same class and same administrative unit

The result of the creation of uncadastred geometries through the ArcGIS worlflow produced a lot a very tiny polygons that would be deleted during the rasterisation of the geometries for the computation of the LC proportions using r.zonal.classes.

To avoid that, here we intersect current geometries produced through the ArcGIS worlflow with the administrative unit polygons and union adjacent geometries (dissolve) with same class and same administrative unit id. The aim is to avoid multiple small neighbouring polygons sharing the same land use.

**Create temporary table with type of land use ('code_walou') and ID of administrative unit**

In this step, a temporary table is created that will contain both type of land use ('code_walou') and ID of administrative unit. Those attributes will be used in the next step to merge and dissolve adjacent polygons sharing the same land use in the same administrative unit. 

In [ ]:
# Create temporary table with uncadastred geometries intersected with administrative boundaries
start_import = start_processing()
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("DROP TABLE IF EXISTS {uncad_schem}.tmp".format(uncad_schem='non_cadastre'))
cursor.execute("CREATE TABLE IF NOT EXISTS {uncad_schem}.tmp AS (\
SELECT a.objectid, a.code_walou, b.comcod,(ST_Dump(ST_Intersection(a.geom, b.geom))).geom AS geom \
FROM {uncad_schem}.{uncad_table} AS a INNER JOIN {uncad_schem}.{admin} AS b \
ON ST_Intersects(a.geom, b.geom))".format(uncad_schem='non_cadastre', 
                                          uncad_table=data['uncadastred_spaces'][0], admin=data['communes_rw'][0]))
con.commit()
con.close()
print_processing_time(start_import, "Process achieved in ")

**Create table with dissolved geometries to avoid multiple small neighbouring polygons sharing the same land use**

In this step, attributes 'code_walou' and 'comcod' are used to merge and dissolve adjacent polygons sharing the same land use in the same administrative unit. 

In [ ]:
# Add index on specific columns
con = create_pg_connexion(config_parameters)
create_index(con, 'non_cadastre', 'tmp', 'code_walou')
create_index(con, 'non_cadastre', 'tmp', 'geom', is_geom=True)
con.close()

In [ ]:
# Name of the dissolved table
dissolved_geom = "%s_dissolved"%data['uncadastred_spaces'][0]
dissolved_geom

In [ ]:
# Create table with dissolved geometries
start_import = start_processing()
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("DROP TABLE IF EXISTS non_cadastre.%s"%dissolved_geom)
cursor.execute("CREATE TABLE IF NOT EXISTS non_cadastre.%s AS (\
SELECT min(objectid) AS objectid, code_walou, comcod, (ST_Dump(ST_Union(geom))).geom AS geom \
FROM non_cadastre.tmp GROUP BY code_walou, comcod)"%dissolved_geom)
con.commit()
con.close()
print_processing_time(start_import, "Process achieved in ")

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
add_serial_pkey(con, 'non_cadastre', dissolved_geom, 'uncadastr_id', pkey=False)

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'non_cadastre', dissolved_geom, row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

In [ ]:
# Drop temporary table 
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("DROP TABLE IF EXISTS non_cadastre.tmp")
con.commit()
con.close()

**Export final uncadastred geometries to GPKG file**

In [ ]:
# Export from PostGIS
ogr2ogr_export(data['uncadastred_spaces_dissolved'][1], config_parameters, 
               schema='non_cadastre', table=dissolved_geom, verbose=False)

# Compute proportion of LC classes for uncadastred geometries (in GRASS)

## Import uncadastred geometries (vector)

In [ ]:
# Create and launch mapset 
launch_mapset("UNCAD")

In [ ]:
# Import vector
start_import = start_processing()
gscript.run_command('v.import', overwrite=True, input=data['uncadastred_spaces_dissolved'][1], 
                    output=data['uncadastred_spaces_dissolved'][0])
print_processing_time(start_import, "Import achieved in ")

In [ ]:
# Rasterize dissolved uncadastred geometries
start = start_processing()
gscript.run_command('g.mapsets', mapset='FUSIONS', operation='add')
gscript.run_command('g.region', vector=data['uncadastred_spaces_dissolved'][0], align=data['landcover_folder'][0])
gscript.run_command('v.to.rast', overwrite=True, input=data['uncadastred_spaces_dissolved'][0], 
                    output=data['uncadastred_spaces_dissolved'][0], use='cat', memory=30000)
print_processing_time(start, "Rasterisation achieved in ")

## Compute LC proportion by uncadastred geom

In [ ]:
# Check if add-on is already installed in the computer and install it not yet installed
check_install_addon("r.zonal.classes")

In [ ]:
### Compute LC class proportions in cadastral plots
# Create mapset 
start = start_processing()
launch_mapset("UNCAD")
gscript.run_command('g.mapsets', quiet=True, mapset='FUSIONS', operation='add')
gscript.run_command('g.region', raster=data['uncadastred_spaces_dissolved'][0], align=data['landcover_folder'][0])
# Compute LC proportions
tmp_csv = "%s_rzonalclasses"%gscript.tempfile() # Path to temporary file output
ouput_csv = data['lc_uncad'][1]
gscript.run_command('r.zonal.classes', overwrite=True, zone_map=data['uncadastred_spaces_dissolved'][0], 
                    raster=data['landcover_folder'][0], prefix='lc', csvfile=tmp_csv)
print_processing_time(start, "Proportions of LC computed in achieved in ")

In [ ]:
### Import the csv output of r.zonal.classes in GRASS SQlite database
start_import = start_processing()
# Load csv content in python dictionnary
incsv = open(tmp_csv, 'r')
reader = csv.reader(incsv, delimiter='|')
header = next(reader)
value_dict = {row[0]:row[1:] for row in reader}
incsv.close()
# Insert SQL
table_name = "lc_prop"
rzonalclasses_sql_insert(table_name, header, value_dict, overwrite=True)
### Create table 'uncad_with_prop' with uncadastred id's linked with lc_prop
table_uncad_with_prop = "uncad_with_prop"
sql_query = gscript.tempfile()
fsql = open(sql_query, 'w')
fsql.write('BEGIN TRANSACTION;\n')
if gscript.db_table_exist(table_uncad_with_prop):
        fsql.write('DROP TABLE %s;\n'%table_uncad_with_prop)
create_statement = 'CREATE TABLE %s AS '%table_uncad_with_prop
create_statement += 'SELECT a.uncadastr_id, b.* FROM %s AS a '%data['uncadastred_spaces_dissolved'][0]
create_statement += 'LEFT JOIN lc_prop AS b ON a.cat=b.cat;\n'
fsql.write(create_statement)
fsql.write('END TRANSACTION;')
fsql.close()
gscript.run_command('db.execute', input=sql_query, quiet=True)
print_processing_time(start_import, "Computation of LC classes proportions achieved in ")

## Export table to csv

In [ ]:
# Export attribute table to csv
if not os.path.exists(os.path.split(data['lc_uncad'][1])[0]):
    os.makedirs(os.path.split(data['lc_uncad'][1])[0])    
gscript.run_command('db.select', overwrite=True, sql="SELECT * FROM %s"%table_uncad_with_prop,
                    output=data['lc_uncad'][1])

## Compute find LC class below geom's centroid from tiny polygons

In [ ]:
# Get list of ID of geometries for which r.zonal.classes failed to compute LC proportion (too tiny - sliver polygons)
start_import = start_processing()
query = 'SELECT uncadastr_id FROM %s WHERE lc_mode is null'%table_uncad_with_prop
sliverpoly_ids = list(gscript.parse_command('db.select', flags='c', sql=query).keys())
count = len(sliverpoly_ids)
print("There are %s polygon(s) tiny and narrowed for which r.zonal.classes did not compute any land cover proportion."%count)

In [ ]:
### Create a new layer with only tiny/narrowed polygons' centroids (on point format)
import math
start_import = start_processing()
# Get centroids of selected polygons
list_temp = []
x = range(0, (math.ceil(len(sliverpoly_ids)/5000)+1)*5000, 5000)
for i,n in enumerate(x[1:],1):
    if n > len(sliverpoly_ids):
        n = len(sliverpoly_ids)
    print("Extracting items from %s to %s"%(x[i-1],n))
    tmp_vect = gscript.tempname(12)
    list_temp.append(tmp_vect)
    gscript.run_command('v.extract', overwrite=True,
                          input=data['uncadastred_spaces_dissolved'][0], type='centroid',
                          where="uncadastr_id in ('%s')"%"','".join(sliverpoly_ids[x[i-1]:n]), 
                          output=tmp_vect)
tmp_centroids = gscript.tempname(12)
gscript.run_command('v.patch', flags='e', overwrite=True, input=','.join(list_temp), output=tmp_centroids)
print("Merging all intermediate vector map together")
# Transform centroids type to point layer
gscript.run_command('v.type', overwrite=True, input=tmp_centroids,
                      output='narrowed_uncad_point', from_type='centroid', to_type='point')
# Drop not needed column
gscript.run_command('v.db.dropcolumn', map='narrowed_uncad_point', columns='objectid,code_walou,comcod')
# Add column to store raster value
gscript.run_command('v.db.addcolumn', map='narrowed_uncad_point', columns='lc_mode integer')
# Use v.what.rast to find LC raster value for the centroid point of the polygon 
gscript.run_command('v.what.rast', map='narrowed_uncad_point',raster='lc', column='lc_mode') 
print_processing_time(start_import, "Extraction of LC raster value bellow tiny/narrowed geometries' centroid achieved in ")

In [ ]:
# Delete temporary files
gscript.run_command('g.remove', flags='f', type='vect', name=','.join(list_temp))
gscript.run_command('g.remove', flags='f', type='vect', name=tmp_centroids)

## Export table to csv

In [ ]:
# Export attribute table to csv
if not os.path.exists(os.path.split(data['lc_uncad_tiny'][1])[0]):
    os.makedirs(os.path.split(data['lc_uncad_tiny'][1])[0])    
gscript.run_command('db.select', overwrite=True, 
                    sql="SELECT * FROM 'narrowed_uncad_point'", output=data['lc_uncad_tiny'][1])

# Import output from r.zonal.classes in Postgis and join with geom

## Import csv output from r.zonal.classes (large polygons)

In [ ]:
# Create a list for the column declaration of the CREATE table statement
with open(data['lc_uncad'][1], 'r') as f:
    header = next(f).split("\n")[0]
colum_declaration = [(col.lower(),'integer') if col in ('uncadastr_id','cat') 
                     else ("lc_mode",'integer') if col == "lc_mode" 
                     else (col,'double precision') for col in header.split("|")]
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['lc_uncad'][1], colum_declaration, 'non_cadastre', 
           data['lc_uncad'][0], delimiter="|", overwrite=False)
# Close connexion to postgres database
con.close()

## Import csv output from v.what.rast (tiny polygons)

In [ ]:
# Create a list for the column declaration of the CREATE table statement
with open(data['lc_uncad_tiny'][1], 'r') as f:
    header = next(f).split("\n")[0]
colum_declaration = [(col.lower(),'integer') if col in ('uncadastr_id','cat') 
                     else ("lc_mode",'integer') for col in header.split("|")]
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['lc_uncad_tiny'][1], colum_declaration, 'non_cadastre', 
           data['lc_uncad_tiny'][0], delimiter="|", overwrite=False)
# Close connexion to postgres database
con.close()

## Join LC proportion information to uncadastred geometries table 

**Join r.zonal.classes results (large polygons)**

In [ ]:
# Rename column containg the information of 'walousmaj'
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("DROP TABLE IF EXISTS %s.%s_lcprop"%('non_cadastre', dissolved_geom))
cursor.execute("CREATE TABLE IF NOT EXISTS {uncad_schem}.{table}_lcprop AS (\
SELECT a.geom, a.{walousmaj} ,b.* FROM {uncad_schem}.{table} AS a LEFT JOIN {uncad_schem}.{lc_prop} AS b \
ON a.uncadastr_id=b.uncadastr_id)".format(uncad_schem='non_cadastre',
                                  walousmaj=data['uncadastred_spaces'][2], table=dissolved_geom,
                                  lc_prop=data['lc_uncad'][0]))

con.commit()
con.close()

**Join v.what.rast results (tiny polygons)**

In [ ]:
# Rename column containg the information of 'walousmaj'
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("UPDATE {uncad_schem}.{table}_lcprop AS a SET lc_mode = b.lc_mode \
FROM {uncad_schem}.{tiny} AS b \
WHERE a.uncadastr_id=b.uncadastr_id".format(uncad_schem='non_cadastre',
                                             tiny=data['lc_uncad_tiny'][0], table=dissolved_geom))

con.commit()
con.close()

**Check if all rows have information on 'lc_mode' column**

In [ ]:
# Query
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("SELECT count(*) FROM %s.%s_lcprop WHERE lc_mode is null"%('non_cadastre', dissolved_geom))
count = int(cursor.fetchone()[0])
con.commit()
con.close()
print("There are %s rows without value for 'lc_mode'."%count)

## Add columns missing

In [ ]:
# Rename column containg the information of 'walousmaj'
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("ALTER TABLE %s.%s_lcprop ADD COLUMN IF NOT EXISTS all_hilucs text[]"%('non_cadastre',dissolved_geom))
cursor.execute("ALTER TABLE %s.%s_lcprop RENAME COLUMN %s TO walousmaj"%('non_cadastre',dissolved_geom,
                                                             data['uncadastred_spaces'][2]))
con.commit()
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'non_cadastre', '%s_lcprop'%dissolved_geom, row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Add attributes for walousmaj (level 1 to 4)

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create columns for each level of the legend 
create_walousmaj_levels(con, result_table_schema='non_cadastre', 
                        result_table_name='%s_lcprop'%dissolved_geom,
                        colum_label="walousmaj")
# Close connexion to postgres database
con.close()

## Add attributes for HilucsLandUse (scenario 1 and 2)

In [ ]:
# Define a list of classes from 'walousmaj' or 'all_hilucs' that should be ignored when creating 'HilucsLandUse'
cl_ignore = []

# Define a list of classes from 'walousmaj' that are not compliant 
# with INSPIRE and should be truncated of one level (e.g., '5_1_A' should become '5_1')
cl_truncate = ['6_6_B']

# Define a lookup table (list of tupples) for convertion of nature conservation labels from walousmaj to INSPIRE HILUCS legend
inspire_lookup = []

# Define a list of Hilucs classes that should be removed if presence of sublevels
# classes (more precise) classes in the array. E.g., is an array contains '4_1' and '4_1_1', the remaining 
# information will be only the most detailed classe which is '4_1_1'
cl_remove = ['4','6','4_1','4_3','6_3']

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create column HilucsLanduse compliant with INSPIRE
create_hilucs_landuse_1(con, result_table_schema='non_cadastre', 
                        result_table_name='%s_lcprop'%dissolved_geom,
                        cl_truncate=cl_truncate, cl_lookup=inspire_lookup,
                        colum_label="hilucslanduse_1")
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create column HilucsLanduse compliant with INSPIRE
create_hilucs_landuse_2(con, result_table_schema='non_cadastre', 
                        result_table_name='%s_lcprop'%dissolved_geom, 
                        cl_ignore=cl_ignore, cl_truncate=cl_truncate, 
                        cl_lookup=inspire_lookup ,cl_remove=cl_remove, colum_label="hilucslanduse_2")
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
whereconditions.append("uncadastr_id = 67780")
df = display_header(con, 'non_cadastre', '%s_lcprop'%dissolved_geom, 
                    where=' AND '.join(whereconditions), row_num=8)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Backup of Postgresql data

**Backup of table with uncadastred areas**

In [ ]:
# Create a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'non_cadastre', 
           '%s_lcprop'%dissolved_geom, data['backup_uncadastred_table'])

# Create new table "cusw2018_all" containing all geometries (cadastred and uncadastred)

In [ ]:
# Set name of table containing all geometries (cadastred and uncadastred)
cusw_all_table='cusw2018_all'

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cusw2018_all table
create_cuswall_table(con, 'results', 'cusw2018', 
                     'non_cadastre', '%s_lcprop'%dissolved_geom, output_table=cusw_all_table)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
whereconditions.append("uncadastr_id = 67780")
df = display_header(con, 'results', cusw_all_table, 
                    where=' AND '.join(whereconditions), row_num=8)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Backup of Postgresql data

**Backup of table with all geometries**

In [ ]:
# Create a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'results', 'cusw2018_all', data['backup_allgeom_table'])


**Backup of the whole database**

In [ ]:
# Create a back-up of the whole database
dump_db(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], data['backup_db'])

# Export from PostgreSQL

## Export table uncadastred spaces in EPSG:31370 (Belge 1972 / Belgian Lambert 72)

In [ ]:
# Export the classification results as shapefile
query_export = "SELECT geom, uncadastr_id, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
array_to_string(all_hilucs,',') as all_hilucs, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 \
FROM non_cadastre.%s_lcprop"%dissolved_geom
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/non_cadastre/uncadastred_spaces_31370.gpkg"
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

## Export table uncadastred spaces in EPSG:3812 (ETRS89 / Belgian Lambert 2008)

In [ ]:
# Export the classification results as shapefile
query_export = "SELECT ST_Transform(geom, 3812) as geom, uncadastr_id, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
array_to_string(all_hilucs,',') as all_hilucs, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 \
FROM non_cadastre.%s_lcprop"%dissolved_geom
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/non_cadastre/uncadastred_spaces_3812.gpkg"
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

## Export table Cusw2018_all in EPSG:31370 (Belge 1972 / Belgian Lambert 72)

In [ ]:
# Export the table as Shapefile
query_export = "SELECT geom, capakey, uncadastr_id, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
array_to_string(all_hilucs,',') as all_hilucs, rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_all_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_31370.shp"%cusw_all_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

In [ ]:
# Export the table as Geopackage
query_export = "SELECT geom, capakey, uncadastr_id, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
array_to_string(all_hilucs,',') as all_hilucs, rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_all_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_31370.gpkg"%cusw_all_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

## Export table Cusw2018_all in EPSG:3812 (ETRS89 / Belgian Lambert 2008)

In [ ]:
# Export the table as Shapefile
query_export = "SELECT ST_Transform(geom, 3812) as geom, capakey, uncadastr_id, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
array_to_string(all_hilucs,',') as all_hilucs, rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_all_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_3812.shp"%cusw_all_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)

In [ ]:
# Export the table as Geopackage
query_export = "SELECT ST_Transform(geom, 3812) as geom, capakey, uncadastr_id, \
lc_mode,lc_prop_1,lc_prop_2,lc_prop_3,lc_prop_4,lc_prop_5,lc_prop_6,lc_prop_7,lc_prop_8,lc_prop_9,lc_prop_80,lc_prop_90, \
array_to_string(all_hilucs,',') as all_hilucs, rulebased_leaf, postclas_rule, \
walousmaj, walousmaj_l1, walousmaj_l2, walousmaj_l3, walousmaj_l4,\
hilucslanduse_1, array_to_string(hilucslanduse_2,',') as hilucslanduse_2 FROM results.%s"%cusw_all_table
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/%s_3812.gpkg"%cusw_all_table
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)